# A8 Alpaca Instuction Tuning Evalutation

## Instruction-Tuning

In [2]:
file_path = "alpaca_data.json"

### Step 1: Load the dataset

In [3]:
# Step 1: Load the dataset
from datasets import Dataset

In [4]:
# Load the JSON file into a dataset
dataset = Dataset.from_json(file_path)
# dataset = Dataset.from_json(file_path, format="json")
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 52002
})

In [5]:
dataset[0]

{'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'instruction': 'Give three tips for staying healthy.'}

In [6]:
# dataset.select(range(100))

### Step 2: Load the model & Tokenizer

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name_or_path = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map = 'auto'
)
tokenizer           = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
# set instruction
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text     = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

# check instruction-prompt
formatting_prompts_func(dataset[:2])

['### Question: Give three tips for staying healthy.\n ### Answer: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 '### Question: What are the three primary colors?\n ### Answer: The three primary colors are red, blue, and yellow.']

In [9]:
# use the DataCollatorForCompletionOnlyLM to train your model on the generated prompts only
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
response_template = "### Answer:"
collator          = DataCollatorForCompletionOnlyLM(response_template, tokenizer = tokenizer)
collator

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


DataCollatorForCompletionOnlyLM(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [10]:
# make sure to pass a correct value for max_seq_len as the default value will be set to min (tokenizer.modle_max_lenght, 1024)
max_seq_length = min(tokenizer.model_max_length, 1024)
max_seq_length

1024

### Task 3: Evaluation

In [11]:
# Step 1: Load the dataset
from datasets import load_dataset
eval_dataset = load_dataset("tatsu-lab/alpaca_eval", split='eval')
eval_dataset

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for tatsu-lab/alpaca_eval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tatsu-lab/alpaca_eval
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['instruction', 'output', 'generator', 'dataset'],
    num_rows: 805
})

train func

In [ ]:
# Train the model
max_seq_length = min(tokenizer.model_max_length, 1024)
trainer = SFTTrainer(
    model,
    train_dataset   = dataset.select(range(100)),
    eval_dataset    = eval_dataset.select(range(50)),
    max_seq_length  = max_seq_length,
    formatting_func = formatting_prompts_func,
    data_collator   = collator,
)

trainer.train()

In [ ]:
trainer.save_model("/model/a8")